# Notebook to Deploy the YOLOv8 Model to a Sagemaker Endpoint

## Import Dependencies

In [1]:
import numpy as np
from ultralytics import YOLO
import os
import boto3
from sagemaker.pytorch import PyTorchModel
from sagemaker.deserializers import JSONDeserializer
import subprocess
import sagemaker
import shutil

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


## Define the parameters

In [2]:
# AWS Account Parameters
session = boto3.Session()

In [3]:
# S3 Parameters: Make sure the bucket exists in you account in S3
bucket_name = "playgrounddatascience"
model_name = 'yolov8l.pt'
prefix = "yolo/model_endpoint"

In [4]:
# Sagemaker Parameters
sagemaker_execution_role = sagemaker.get_execution_role()
INSTANCE_TYPE = 'ml.m5.2xlarge'
ENDPOINT_NAME = 'yolo-endpoint'

## Download YOLOv8 Model File

In [5]:
# Check if the file already exists
if os.path.isfile(os.path.join("../", model_name)):
    print(f'Model file already exists at location: {os.path.join("../", model_name)}')
else: 
    print(f'Downloading {model_name} file ...')
    YOLO(model_name)
    shutil.move(model_name, os.path.join("../", model_name))
    print(f'Moved {model_name} file to root of project')

100%|██████████| 83.7M/83.7M [00:00<00:00, 213MB/s]


Moved yolov8l.pt file to root of project


In [6]:
# Compress the model to be uploaded to S3
try:
    bashCommand = "tar -czvf ../model.tar.gz ../code/ ../yolov8l.pt"
    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    print(f'Files Compressed Successfully !!!')
    os.remove(os.path.join("../", model_name))
    print(f'Unwanted File ({model_name}) Removed Successfully !!!')
except:
    print('Failure in Compressing File !!!')

tar: Removing leading `../' from member names


Files Compressed Successfully !!!
Unwanted File (yolov8l.pt) Removed Successfully !!!


## Upload Model Files to S3

In [7]:
# Test if session is properly created. This will print a list of buckets in your account, if you have permissions
s3_client = session.resource('s3')
for bucket in s3_client.buckets.all():
    print(bucket.name)
    break

amplify-suruapp-dev-155600-deployment


In [8]:
# Upload the compressed file to S3
try:
    s3_client.meta.client.upload_file('../model.tar.gz', bucket_name, prefix + "/" + 'model.tar.gz')
    print(f'Succesfully Uploaded Model Files to S3 at {prefix}')
except:
    print('Failure in Uploading File to S3 !!!')

Succesfully Uploaded Model Files to S3 at yolo/model_endpoint


## Create and Deploy the Model on Sagemaker Endpoint

In [9]:
model = PyTorchModel(entry_point='inference.py',
                     model_data='s3://' + bucket_name  + '/' + prefix + '/model.tar.gz', 
                     framework_version='1.12', 
                     py_version='py38',
                     role=sagemaker_execution_role,
                     env={'TS_MAX_RESPONSE_SIZE':'20000000', 'YOLOV8_MODEL': model_name})

In [10]:
predictor = model.deploy(initial_instance_count=1, 
                         instance_type=INSTANCE_TYPE,
                         deserializer=JSONDeserializer(),
                         endpoint_name=ENDPOINT_NAME)

----!

## Delete Endpoint and Configuration

In [11]:
# set to 1, if you want to delete the sagemaker endpoint and configuration
delete_sagemaker_entities = 0

In [12]:
if delete_sagemaker_entities:
    sm_client = boto3.client(service_name="sagemaker")
    response = sm_client.describe_endpoint_config(EndpointConfigName=ENDPOINT_NAME)
    endpoint_config_name = response['EndpointConfigName']
    try:
        sm_client.delete_endpoint(EndpointName=ENDPOINT_NAME)
        print(f'Endpoint {ENDPOINT_NAME} deleted successfully !!!')
    except:
        print(f'ERROR, in deleting Endpoint {ENDPOINT_NAME} !!!')
    try:
        sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
        print(f'Endpoint {ENDPOINT_NAME} configuration deleted successfully !!!')
    except:
        print(f'ERROR, in deleting Endpoint {ENDPOINT_NAME} configuration !!!')

## Delete Unwated Files and Folders

In [13]:
root = "./"
for path, subdirs, files in os.walk(root):
    for name in files:
        file_path = os.path.join(path, name)
        if "__pycache__" in file_path or ".ipynb_checkpoints" in file_path or ".DS_Store" in file_path:
            os.remove(file_path)